In [1]:
import pandas as pd 
import numpy as np
import os
from PIL import Image
import torch
from skimage import io, transform
from sklearn.metrics import classification_report, auc, roc_auc_score
import warnings
warnings.filterwarnings('ignore')

from models.model_color import load_color_model, COLORS
from models.model_category import load_category_model, CATEGORIES
from models.model_door import load_door_model, DOORS
from preprocess.detect_car import car_image

DATA_PATH = '/home/misho/Uni/Vision/YOLO/keras-YOLOv3-model-set/Myauto_data/Car_Images/'
IMAGES_PATH = DATA_PATH + '{}/{}'

In [2]:
test_data = pd.read_csv('./test_data.csv').sample(frac=1)
test_data.dropna(inplace=True)
test_data = test_data[:1000]

color_model = load_color_model()
door_model = load_door_model()
category_model = load_category_model()

In [3]:
def predict(ID, img_pathes, model):
    images = []
    for img in img_pathes:
        image = transform.resize(io.imread(IMAGES_PATH.format(ID, img)), (224, 224))/255
        image = image[:,:,:3]
        images.append(torch.Tensor(np.einsum('ijk->kij',image)))
    inp = torch.stack(images)
    out_color = color_model(inp)
    out_color = torch.nn.Softmax()(torch.sum(out_color, dim=0))
    out_color = np.array(out_color.detach()).argmax()
    
    out_door = door_model(inp)
    out_door = torch.nn.Softmax()(torch.sum(out_door, dim=0))
    out_door = np.array(out_door.detach()).argmax()
    
    out_category = category_model(inp)
    out_category = torch.nn.Softmax()(torch.sum(out_category, dim=0))
    out_category = np.array(out_category.detach()).argmax()
    return out_color, out_door, out_category


def get_car_images(ID):
    try:
        img_ls = os.listdir(DATA_PATH + str(ID))
    except:
        return []
    car_imgs = [car_image(np.array(Image.open(IMAGES_PATH.format(ID, img)))) for img in img_ls]
    result = []
    for img, car in zip(img_ls, car_imgs):
        if car:
            result.append(img)
    return result


def get_color_index(a):
    try:
        index = COLORS.index(a)
        return index
    except:
        return len(COLORS)

def get_door_index(a):
    try:
        index = DOORS.index(a)
        return index
    except:
        return len(DOORS)


def get_category_index(a):
    if a in ['Minivan', 'Microbus']:
        a = 'Microbus'
    if a in ['Hatchback', 'Universal']:
        a = 'Hatchback'
    if a in ['Coupe', 'Cabriolet']:
        a = 'Coupe'
    try:
        index = CATEGORIES.index(a)
        return index
    except:
        return len(CATEGORIES)

In [4]:
colors, doors, categories = [],[],[]
for i in range(len(test_data)):
    row = test_data.iloc[i]
    car_img_pathes = get_car_images(row.ID)
    if len(car_img_pathes) == 0:
        colors.append(None)
        doors.append(None)
        categories.append(None)
        continue
    color, door, category = predict(row.ID, car_img_pathes[:1], color_model)
    colors.append(color)
    doors.append(door)
    categories.append(category)
    

test_data['color_answer'] = test_data.Color.apply(get_color_index)
test_data['color_prediction'] = colors
test_data['door_answer'] = test_data.Doors.apply(get_door_index)
test_data['door_prediction'] = doors
test_data['category_answer'] = test_data.Category.apply(get_category_index)
test_data['category_prediction'] = categories

In [5]:
test_data.dropna(inplace=True)
test_data

,ID,Doors,Color,Category,color_answer,color_prediction,door_answer,door_prediction,category_answer,category_prediction
4170,45733858,4/5,Silver,Jeep,3,3.0,1,1.0,1,1.0
7356,45707100,4/5,Black,Sedan,5,5.0,1,1.0,3,3.0
8547,45696179,4/5,Black,Sedan,5,5.0,1,1.0,3,3.0
6221,45716815,4/5,Black,Minivan,5,5.0,1,1.0,2,1.0
4588,45732031,4/5,Silver,Jeep,3,3.0,1,1.0,1,1.0
...,...,...,...,...,...,...,...,...,...,...
2864,45738881,4/5,Black,Hatchback,5,0.0,1,1.0,5,2.0
11443,44703853,2/3,Red,Coupe,2,2.0,0,0.0,4,3.0
13555,45654552,4/5,Silver,Hatchback,3,3.0,1,1.0,5,0.0
13866,45596171,4/5,Grey,Jeep,4,0.0,1,1.0,1,1.0


results on color prediction

In [7]:
print(classification_report(list(test_data.color_answer), list(test_data.color_prediction), target_names = COLORS+['OUT_OF_CONSIDERATION']))

                      precision    recall  f1-score   support

                Blue       0.65      0.78      0.71        79
               White       0.96      0.90      0.93       209
                 Red       0.52      1.00      0.68        26
              Silver       0.77      0.87      0.82       174
                Grey       0.74      0.71      0.72       107
               Black       0.81      0.91      0.86       228
               Green       0.00      0.00      0.00        16
OUT_OF_CONSIDERATION       0.00      0.00      0.00        57

            accuracy                           0.79       896
           macro avg       0.56      0.65      0.59       896
        weighted avg       0.74      0.79      0.76       896



results on door number prediction

In [10]:
print(classification_report(list(test_data.door_answer), list(test_data.door_prediction), target_names = DOORS+['>5']))

              precision    recall  f1-score   support

         2/3       0.26      0.62      0.37        29
         4/5       0.98      0.94      0.96       861
          >5       0.00      0.00      0.00         6

    accuracy                           0.93       896
   macro avg       0.41      0.52      0.44       896
weighted avg       0.95      0.93      0.94       896



results on category prediction

In [12]:
print(classification_report(list(test_data.category_answer), list(test_data.category_prediction), target_names = CATEGORIES+['OUT_OF_CONSIDERATION']))

                      precision    recall  f1-score   support

           HatchBack       0.00      0.00      0.00         0
                Jeep       0.83      0.83      0.83       265
            Microbus       0.32      0.82      0.46        28
               Sedan       0.89      0.68      0.77       393
               Coupe       0.00      0.00      0.00        17
OUT_OF_CONSIDERATION       0.00      0.00      0.00       193

            accuracy                           0.57       896
           macro avg       0.34      0.39      0.34       896
        weighted avg       0.64      0.57      0.60       896



# Code for testing model results on user input

In [ ]:
# CSV file name, that contains ID column with car IDs
CSV_FILE = ''
# Path of image directory
DATA_PATH = ''

IMAGES_PATH = DATA_PATH + '{}/{}'

## This code will generate color, door and category predictions dor each ID by adding separate columns in the csv file for each of those predictions

In [ ]:
data = pd.read_csv(CSV_FILE)
colors, doors, categories = [],[],[]
for i in range(len(test_data)):
    row = test_data.iloc[i]
    car_img_pathes = get_car_images(row.ID)
    if len(car_img_pathes) == 0:
        colors.append(None)
        doors.append(None)
        categories.append(None)
        continue
    color, door, category = predict(row.ID, car_img_pathes[:1], color_model)
    colors.append(color)
    doors.append(door)
    categories.append(category)
    
colors, doors, categories = [],[],[]
for i in range(len(test_data)):
    row = test_data.iloc[i]
    car_img_pathes = get_car_images(row.ID)
    if len(car_img_pathes) == 0:
        colors.append(None)
        doors.append(None)
        categories.append(None)
        continue
    color, door, category = predict(row.ID, car_img_pathes[:1], color_model)
    colors.append(color)
    doors.append(door)
    categories.append(category)
    

test_data['color_prediction'] = colors
test_data['door_prediction'] = doors
test_data['category_prediction'] = categories

test_data.to_csv('output.csv',index=False)